In [2]:
# import libraries
import os
import pymongo
import webbrowser
import dns
import pandas as pd

In [3]:
# connect to mongoDB
client=pymongo.MongoClient("mongodb://localhost:27017/")

In [4]:
#create database
mydb= client["Health"]
#hospital collection
coll1=mydb["Hospital"]
#create doctor's collection
coll2= mydb["Doctor"]
#create patients collection
coll3=mydb["Patients"]

In [5]:
def collections():
    #documents for hospital
    data1 =[
            {"hospital_id":1, "hospital_name":'Kenyatta', 'bed-count': 250, 'address':'upperhill'},
            {"hospital_id":2, "hospital_name":'Nairobi', 'bed-count': 500, 'address':'lngt Rd'},
            {"hospital_id":3, "hospital_name":'Neema', 'bed-count': 800, 'address':'thika Rd'},
            {"hospital_id":4, "hospital_name":'St Mary', 'bed-count': 456, 'address':'kas av'},
            {"hospital_id":5, "hospital_name":'Medanta', 'bed-count': 340, 'address':'Waiyaki way'},
            {"hospital_id":6, "hospital_name":'Nairobi west', 'bed-count': 678 , 'address':'south c'}
    ]
            
    #insert data into Hospital collection
    doc1=coll1.insert_many(data1)
    #Doctors collection documents
    data2=[
        {"doctor_id":7, 'doctor_name':"MaryAnne", "hospital_id":1, "date_joined":"2016-07-12", "Speciality":"Gynacologist", "Salary":87000,"experience":5},
        {"doctor_id":8, 'doctor_name':"Migot", "hospital_id":3, "date_joined":"2018-08-23", "Speciality":"Paediatric", "Salary": 98000,"experience":3},
        {"doctor_id":9, 'doctor_name':"Lucy", "hospital_id":4, "date_joined":"2011-02-02", "Speciality":"Nuero", "Salary": 300000,"experience":10},
        {"doctor_id":10, 'doctor_name':"Esther", "hospital_id":5, "date_joined":"2015-09-15", "Speciality":"Nurse", "Salary":90000,"experience":6},
        {"doctor_id":11, 'doctor_name':"John", "hospital_id":6, "date_joined":"2013-11-09", "Speciality":"MOH", "Salary":120000,"experience":8}
    ]
    #insert record into doctors collection
    #coll2.insert_many(data2)
    doc2=coll2.insert_many(data2)
    #patients collection documents
    data3= [
        {"patient_names": "Karu shuli", "date_birth":"1995-11-24","inpatient":True,"gender":"M", "doctor_id":9,"hospital_id":4},
        {"patient_names": "Daniel kioko", "date_birth":"1989-4-12","inpatient":False,"gender":"M", "doctor_id":11,"hospital_id": 6},
        {"patient_names": "Nasra Hassan", "date_birth":"1997-12-8","inpatient":True,"gender":"F", "doctor_id":10,"hospital_id":5},
        {"patient_names": "Jane K", "date_birth":"1990-1-18","inpatient":True,"gender":"F", "doctor_id":9,"hospital_id":4},
        {"patient_names": "Jolyne G", "date_birth":"1995-3-14","inpatient":False,"gender":"F", "doctor_id":7,"hospital_id":1},
        {"patient_names": "Nush Rashid", "date_birth":"1993-7-16","inpatient":True,"gender":"M", "doctor_id":8,"hospital_id":3}
    ]
    doc3 = coll3.insert_many(data3)

In [6]:
#collections()

In [7]:
# convert the  hospital dict to pandas dataframe
hosp_df = pd.DataFrame(list(coll1.find()))
# removed the automatically generated _id
hosp_df.drop("_id", inplace= True, axis=1)
# Create a MultiIndex using columns ‘id’ and ‘name’
hosp_df.set_index(["hospital_id", "hospital_name"], inplace=True)
hosp_df

,,address,bed-count
hospital_id,hospital_name,,
1,Kenyatta,upperhill,250
2,Nairobi,lngt Rd,500
3,Neema,thika Rd,800
4,St Mary,kas av,456
5,Medanta,Waiyaki way,340
6,Nairobi west,south c,678


In [8]:
doc_df = pd.DataFrame(list(coll2.find()))
#drop the autmatically generated _id
doc_df.drop('_id', inplace= True, axis=1)
# create multi index using using doctor_id and naame
doc_df.set_index(['doctor_id',  'doctor_name'], inplace= True)
doc_df

,,Salary,Speciality,date_joined,experience,hospital_id
doctor_id,doctor_name,,,,,
7,MaryAnne,87000,Gynacologist,2016-07-12,5,1
8,Migot,98000,Paediatric,2018-08-23,3,3
9,Lucy,300000,Nuero,2011-02-02,10,4
10,Esther,90000,Nurse,2015-09-15,6,5
11,John,120000,MOH,2013-11-09,8,6


In [9]:
#calculate total salary
doc_df[['Salary']].sum()

Salary    695000
dtype: int64

In [10]:
# descriptive statistics for all
stats_Salary = doc_df[['Salary']].describe().astype (int)
stats_Salary

,Salary
count,5
mean,139000
std,90923
min,87000
25%,90000
50%,98000
75%,120000
max,300000


In [11]:
# patients data frame
patients_df = pd.DataFrame(list(coll3.find()))
#index using the patient names
patients_df.set_index("patient_names", inplace = True)
#drop the _id column which is automatically generated in mongoDb collection
patients_df.drop("_id", inplace = True, axis = 1)
patients_df

,date_birth,doctor_id,gender,hospital_id,inpatient
patient_names,,,,,
Karu shuli,1995-11-24,9,M,4,True
Daniel kioko,1989-4-12,11,M,6,False
Nasra Hassan,1997-12-8,10,F,5,True
Jane K,1990-1-18,9,F,4,True
Jolyne G,1995-3-14,7,F,1,False
Nush Rashid,1993-7-16,8,M,3,True


In [22]:
#join first 2 collections
join = mydb.Hospital.aggregate(
[
    {
                "$lookup": {
                "from": "Doctor",
                "localField": "hospital_id",
                "foreignField": "hospital_id",
                "as": "two_join"
                }
            },
    {   "$unwind":"$two_join" },
         
           {   
                "$project":{
                    "_id":0,
                    "hospital_id":1,
                    "address":1,
                    "hospital_name":1,
                    "Speciality":"$two_join.Speciality",
                    "doctor_name":"$two_join.doctor_name"
            
        } 
    }
    
])
join_df = pd.DataFrame(list(join))
join_df

,Speciality,address,doctor_name,hospital_id,hospital_name
0,Gynacologist,upperhill,MaryAnne,1,Kenyatta
1,Paediatric,thika Rd,Migot,3,Neema
2,Nuero,kas av,Lucy,4,St Mary
3,Nurse,Waiyaki way,Esther,5,Medanta
4,MOH,south c,John,6,Nairobi west


In [12]:

#def join the three collections
aggr= mydb.patients.aggregate(
    [
        {
                "$lookup": {
                "from": "Hospital",
                "localField": "hospital_id",
                "foreignField": "hospital_id",
                "as": "hospital_join"
                }
            },
        
{ "$unwind":"$hospital_join" },
          {
              "$lookup":{
              "from": "Doctor", 
              "localField": "hospital_id", 
              "foreignField": "hospital_id",
              "as": "Hospital_join"
        }
    },
         
 { "$unwind":"$hospital_join" },
          {
              "$lookup":{
              "from": "Doctor", 
              "localField": "hospital_id", 
              "foreignField": "hospital_id",
              "as": "Health_join"
        }
    },
         {   "$unwind":"$Health_join" },
         
           {   
                "$project":{
                    "_id":0,
                    "patient_id":1,
                    "patient_name":1,
                    "hospital_name":"$Hospital_join.hospital_name",
                    "doctor_name":"$Health_join.doctor_name",
                    "Speciality":"$Health_join.Speciality"     
            
        } 
    }
    
    ]
)


health_df = pd.DataFrame (list(aggr))
#arrange the collumn to start with the hospital_id
#health_df.set_index("hospital_id", inplace = True)
health_df

""


In [23]:
#convert all the needed dataframes to html
h= hosp_df.to_html()
d=doc_df.to_html()
p=patients_df.to_html()
j=join_df.to_html()
#create html
def html():
    f = open('hrec.html', 'w')

    content = '''<!DOCTYPE html>
    <html>
    <head>
    <title>Health service in Nairobi </title>
    </head>
    
    <body>  
        <h1>Health Report by Ibrae:</h1>
        <thead><h2> Hospital</h2></thead>
        %s
       
        <th><h2>Doctors</h2></th>
        %s

        <thead><caption><h2>Patients</h2></caption></thead>
        %s
    
        <thead><caption><h2>Joined Collections</h2></caption></thead>
        %s
      
    </body>
    <link rel='stylesheet' href='Health.css'/>
    </html>''' % (h, d, p, j) 

    
    f.write(content)
    f.close()
    
    webbrowser.open_new_tab('hrec.html')


html()